In [1]:
from utils.agents import *
from utils.embedding import *
from utils.json_extract import *
from utils.R import *

In [ ]:
print(1)

In [ ]:
print(1)

### Building a Knowledge Graph from a txt

#### Here, we use the paper as the material, refer to [India's Opportunity to Address Human Resource Challenges](https://pubmed.ncbi.nlm.nih.gov/37448434/#:~:text=The%20paucity%20of%20skilled%20personnel%20must%20be%20addressed,budget%20offers%20an%20unprecedented%20opportunity%20to%20do%20this)

In [2]:
try:
    with open('./prompts/graph.txt', 'r', encoding='utf-8') as file:
        KG_task  = file.read()
except FileNotFoundError:
    res = "File not found."

api_key = "3a96f3438411dfd81700ce172c4e5b89.fSLyWokcvYectqvI" # You should put your API_KEY HERE
agent_KG = Agent(api_key=api_key, model_name="glm-4-long", system_content=KG_task )

In [3]:
file_path = "./datasets/cureus-0015-00000040274.txt"

# Opening and reading the file
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        document_content = file.read()
except FileNotFoundError:
    document_content = "The specified file path was not found."

extracted_content = agent_KG.get_response(user_content=document_content)['response_content']
extracted_json = clean_and_extract_json(extracted_content)

In [4]:
# Store this json
output_path = './knowledge_graph/cureus-0015-00000040274.json'

# Writing the `extracted_json` to the specified file
try:
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(extracted_json, json_file, ensure_ascii=False, indent=4)
    res = "File saved successfully."
except Exception as e:
    res = f"An error occurred: {e}"

### Retrieve From this KG using a query

In [8]:
file_path = './knowledge_graph/cureus-0015-00000040274.json'
with open(file_path, 'r', encoding='utf-8') as file:
    extracted_json = json.load(file)

In [9]:
query = 'How can the National Medical Council (NMC) regulate the growing divide between the public and private health sectors in India, particularly in light of the increasing reliance on Auxiliary Nurse Midwives (ANMs) and Accredited Social Health Activists (ASHAs) for rural healthcare delivery, while addressing the broader challenges of digital technology adoption and health infrastructure development as outlined in the National Health Mission (NHM)?'
beta_1 = 3
alpha_1 = 1
alpha_2 = 2
retreival_res = R(query, extracted_json, alpha_1, alpha_2, beta_1)
retreival_res = json.dumps(retreival_res, ensure_ascii=False, indent=4)

In [10]:
retreival_res

'{\n    "concepts_id/name/explanation_and_similarity_sorted": [\n        {\n            "id": 8,\n            "name": "National Medical Council (NMC)",\n            "explanation": "Replaced Medical Commission, aims to improve regulation of health professionals.",\n            "similarity_score": 0.7361299395796563\n        },\n        {\n            "id": 0,\n            "name": "India\'s health indicators",\n            "explanation": "Improved but lag behind peer nations.",\n            "similarity_score": 0.6073716307823804\n        },\n        {\n            "id": 4,\n            "name": "National Health Mission (NHM)",\n            "explanation": "Aims to increase healthcare services, focuses on primary healthcare.",\n            "similarity_score": 0.7193918994073849\n        },\n        {\n            "id": 5,\n            "name": "Accredited Social Health Activists (ASHA)",\n            "explanation": "Female community health workers, serve to increase reach of healthcare.",\n 

### Construct multi agents provided for users

In [11]:
file_path = './prompts/'
dirs = ['agent1.txt', 'agent2.txt', 'agent3.txt', 'agent4.txt', 'fusion.txt']

# Reading the content of files and storing them in the 'tasks' dictionary directly
tasks = {key: open(f"{file_path}{key}", 'r', encoding='utf-8').read() for key in dirs}

In [12]:
file_path = './prompts/'
dirs = ['agent1.txt', 'agent2.txt', 'agent3.txt', 'agent4.txt', 'fusion.txt']

# Reading the content of files and storing them in the 'tasks' dictionary directly
tasks = {key: open(f"{file_path}{key}", 'r', encoding='utf-8').read() for key in dirs}

# Renaming the keys in the 'tasks' dictionary
tasks = {
    'concepts': tasks['agent1.txt'],
    'hint': tasks['agent2.txt'],
    'assess': tasks['agent3.txt'],
    'questions': tasks['agent4.txt'],
    'fusion': tasks['fusion.txt']
}

agents = {}
api_key = "531e9c858c9d4dc1d9295dbf0cdf20e5.zjSejRW3PHh8015f"
llms_list = ["glm-4", "glm-4-plus", "glm-4-long", "glm-4-0520", "glm-4-flash"]

# Loop to initialize Agent instances for each LLM and task content, and create a dictionary with LLM+task as keys
for i in llms_list:
    for task_name, task_content in tasks.items():
        key = f"{i}_{task_name}"  # Create a key by combining LLM model name and task name
        agents[key] = Agent(api_key=api_key, model_name=i, system_content=task_content)

#### Testing an Agent

In [13]:
a0 = agents["glm-4-plus_concepts"]
user_input = """
What does the National Medical Council (NMC) mean?
"""
res0 = a0.get_response(user_content=user_input)['response_content']
print("The reponse is" + " \n" + res0)

The reponse is 
To provide a detailed explanation of the term "National Medical Council (NMC)," I will use the provided external knowledge graph (KG). Let's assume the KG contains relevant information about the NMC.

### External Knowledge Graph (KG):

```json
{
  "nodes": [
    {
      "id": 0,
      "name": "National Medical Council (NMC)",
      "explanation": "The National Medical Council is a regulatory body in India that oversees the medical education and practice, replacing the Medical Council of India (MCI). It is responsible for establishing standards for medical education, granting recognition to medical qualifications, and maintaining a register of medical practitioners."
    },
    {
      "id": 1,
      "name": "Medical Council of India (MCI)",
      "explanation": "The former regulatory body in India for medical education and practice, which was replaced by the National Medical Council (NMC)."
    },
    {
      "id": 2,
      "name": "medical education",
      "explanati

### Choose 5 agents

In [14]:
agent1 = agents["glm-4-flash_concepts"]  # Responsible for handling the 'concepts' task using the 'glm-4-flash' model
agent2 = agents["glm-4-plus_hint"]       # Handles the 'hint' task using the 'glm-4-plus' model
agent3 = agents["glm-4-plus_assess"]     # Manages the 'assess' task using the 'glm-4-plus' model
agent4 = agents["glm-4-0520_questions"]  # Processes the 'questions' task using the 'glm-4-0520' model
agent5 = agents["glm-4-flash_fusion"]    # Takes care of the 'fusion' task using the 'glm-4-flash' model

### Generate responses and Fuse them

In [16]:
agents_parallel = [agent1, agent2, agent3]

input = 'my question is: /n' + query +  \
'please answer my question based on the kwnoledge graph content as follows' + \
retreival_res

response = ""

for agent in agents_parallel:
    res = agent.get_response(user_content=input)['response_content']
    response += res

In [17]:
response

'The concepts and explanations are given as follows:\n{\n  "term0": {\n    "term": "National Medical Council (NMC)",\n    "explanation": "The National Medical Council (NMC) is an organization that replaced the Medical Commission and aims to improve the regulation of health professionals."\n  },\n  "term1": {\n    "term": "National Health Mission (NHM)",\n    "explanation": "The National Health Mission (NHM) is an initiative that aims to increase healthcare services and focuses on primary healthcare."\n  },\n  "term2": {\n    "term": "Accredited Social Health Activists (ASHA)",\n    "explanation": "Accredited Social Health Activists (ASHA) are female community health workers designed to increase the reach of healthcare in rural areas."\n  },\n  "term3": {\n    "term": "Auxiliary Nurse Midwives (ANM)",\n    "explanation": "Auxiliary Nurse Midwives (ANM) are healthcare workers serving populations of 5,000 to 20,000 people, providing essential healthcare services."\n  },\n  "term4": {\n   

In [18]:
answer_fusion = agent5.get_response(user_content=response)['response_content']
print(answer_fusion)

Based on your question, here is the information I have gathered:

1. Extracted Terminology/Concepts and Explanations:
   - National Medical Council (NMC): An organization that replaced the Medical Commission, aiming to improve the regulation of health professionals.
   - National Health Mission (NHM): An initiative focusing on increasing healthcare services, particularly primary healthcare.
   - Accredited Social Health Activists (ASHA): Female community health workers designed to increase the reach of healthcare in rural areas.
   - Auxiliary Nurse Midwives (ANM): Healthcare workers serving populations of 5,000 to 20,000 people, providing essential healthcare services.
   - Digital technology adoption: The integration of digital technologies into healthcare services to improve efficiency, accessibility, and quality of care.
   - Health infrastructure development: The construction and improvement of healthcare facilities such as hospitals, clinics, and other medical infrastructure.

2.

In [19]:
# guide user next query
guide = agent4.get_response(user_content=input)['response_content']

In [20]:
print(guide)

A refined question that makes your query clearer and more questions provided for your future thought are given as follows:

Clarified Question:
"How can the National Medical Council (NMC) effectively regulate and bridge the gap between the public and private health sectors in India, particularly concerning the increasing reliance on Auxiliary Nurse Midwives (ANMs) and Accredited Social Health Activists (ASHAs) for rural healthcare, while concurrently addressing the challenges of adopting digital technologies and developing health infrastructure as outlined under the National Health Mission (NHM)?"

Proposed Guiding Questions:
1. "In what specific ways can the NMC utilize its regulatory powers to ensure equitable healthcare delivery between the public and private sectors, especially in rural areas served by ANMs and ASHAs?"
2. "How can the integration of digital technologies in rural healthcare, as facilitated by the NMC, improve health outcomes and potentially reduce the divide between

#### The asnwer above is more comprehensive; but more tunning will go on...